In [1]:
import nltk
import numpy as np
import random
import string 
import spacy
import requests
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

In [24]:
#data acquisition from web page url
urls = [
        "https://www.bbc.com/news/world-europe-60506682", "https://news.sky.com/story/ukraine-war-the-site-where-hundreds-of-russians-were-killed-after-botched-attempt-to-cross-donets-river-12617958",
        "https://war.ukraine.ua/russia-war-crimes/", "https://www.cfr.org/global-conflict-tracker/conflict/conflict-ukraine", "https://www.bloomberg.com/graphics/2022-ukraine-russia-us-nato-conflict/",
        "https://nytimes.com/interactive/2022/world/europe/ukraine-maps.html", "https://www.msf.org/providing-medical-and-mental-health-treatment-ukraine%E2%80%99s-metro", 
        "https://www.unesco.org/en/ukraine-war/actions-timeline", "https://www.aljazeera.com/news/2022/4/1/ukrainian-cultural-sites-heritage-damaged-russian-invasion-unesco",
        "https://www.aljazeera.com/news/2022/4/1/ukrainian-cultural-sites-heritage-damaged-russian-invasion-unesco", "https://en.wikipedia.org/wiki/Russo-Ukrainian_War",
        "https://en.wikipedia.org/wiki/2022_Russian_invasion_of_Ukraine", "https://blogs.lse.ac.uk/usappblog/2022/03/04/russias-war-with-ukraine-asking-the-right-questions/", 
        "https://www.thejournal.ie/ukraine-russia-questions-answered-5699581-Mar2022/", "https://cnsmaryland.org/2022/03/16/the-war-in-ukraine-questions-and-answers/"
        ]
url = "https://en.wikipedia.org/wiki/Russo-Ukrainian_War"
data = requests.get(url)

# loading the page content into soup
soup = BeautifulSoup(data.content, "html.parser")

# removing the html, style and script tags
for script in soup(["script", "style"]):
    script.extract()

# extrating only the text content 
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
# chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
# text = '\n'.join(chunk for chunk in chunks if chunk)

In [25]:
nlp = English()
# Create the pipeline 'sentencizer' component
sdb = nlp.create_pipe("sentencizer")
nlp.add_pipe(sdb)

for url in urls:
  data = requests.get(url)
  # loading the page content into soup
  soup = BeautifulSoup(data.content, "html.parser")

  # removing the html, style and script tags
  for script in soup(["script", "style"]):
      script.extract()

  # extrating only the text content 
  content = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in content.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text += '\n'.join(chunk for chunk in chunks if chunk)

doc = nlp(text)

sents = []
# getting the sentences from each doc
for sent in doc.sents: 
  sents.append(str(sent))

In [ ]:
for sent in sents:
  print(sent)

In [27]:
# new
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
text = ""
for sent in sents:
  text += sent

In [29]:
raw = text.lower()

stopWords = set(stopwords.words("english"))
sentTokens = nltk.sent_tokenize(raw)
wordTokens = nltk.word_tokenize(raw)

In [30]:
filteredSentence = [w for w in wordTokens if not w.lower() in stopWords]
filteredSentence = []

for w in wordTokens:
  if w not in stopWords:
    filteredSentence.append(w)

In [31]:
print(filteredSentence)

['russo-ukrainian', 'war', '-', 'wikipedia', 'russo-ukrainian', 'war', 'wikipedia', ',', 'free', 'encyclopedia', 'jump', 'navigation', 'jump', 'search', 'ongoing', 'war', 'russia', 'ukraine', 'since', '2014', 'article', 'ongoing', 'war.for', 'wars', 'russia', 'historical', 'ukrainian', 'states', ',', 'see', 'list', 'wars', 'russia', 'ukraine.for', 'recent', 'large-scale', 'invasion', ',', 'see', '2022', 'russian', 'invasion', 'ukraine', '.', 'russo-ukrainian', 'warpart', 'post-soviet', 'conflictsthe', 'military', 'situation', '19', 'may', '2022', '[', 'update', ']', ',', '2022', 'russian', 'invasion', 'ukraine', 'controlled', 'ukraine', 'occupied', 'russia', 'pro-russian', 'forces', 'detailed', 'map', ',', 'see', 'russo-ukrainian', 'war', 'detailed', 'mapdate20', 'february', '2014', '[', 'b', ']', '–', 'present', '(', '8', 'years', ',', '3', 'months', '5', 'days', ')', 'russian', 'invasion', 'ukraine', ':', '24', 'february', '2022', '–', 'present', '(', '3', 'months', '1', 'day', ')', 

In [32]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemmatizeTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemmNormalize(text):
  return LemmatizeTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [33]:
greetingInputs = ("hello", "hi", "greetings", "sup", "what's up","hey",)
greetingResponses = ["Hi", "Hey", "Hi there", "Hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in greetingInputs:
            return random.choice(greetingResponses)

In [34]:
def response(user_response):
    robo_response = ""
    sentTokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemmNormalize, stop_words="english")
    tfidf = TfidfVec.fit_transform(sentTokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sentTokens[idx]
        return robo_response

In [ ]:
flag=True
print("Barsik: My name is Barsik. I will answer your queries related to Ukraine-Russia War. If you want to exit, type Bye!")
while(flag==True):
    userResponse = input()
    userResponse = userResponse.lower()
    if(userResponse != 'bye'):
        if(userResponse == 'thanks' or userResponse == 'thank you' ):
            flag=False
            print("Barsik: You are welcome..")
        else:
            if(greeting(userResponse)!=None):
                print("Barsik: "+greeting(userResponse))
            else:
                print("Barsik: ",end="")
                print(response(userResponse))
                sentTokens.remove(userResponse)
    else:
        flag=False
        print("Barsik: Bye! take care..")

Barsik: My name is Barsik. I will answer your queries related to Ukraine-Russia War. If you want to exit, type Bye!
hello
Barsik: Hi there
donbass
Barsik: 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


[85]
2014–2015 war in donbas
main article: war in donbas
see also: combatants of the war in donbas, list of equipment used by russian separatist forces of the war in donbas, and timeline of the war in donbas
pro-russian unrest
main article: 2014 pro-russian unrest in ukraine
the first protests across southern and eastern ukraine were largely native expressions of discontent with the new ukrainian government.
azov
Barsik: 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


mariupol
melitopol
kyiv
sea of azov
ukr.
azov batalion
Barsik: 

...

mariupol
melitopol
kyiv
sea of azov
ukr.
mariupol 
Barsik: 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


mariupol
melitopol
evacuations of mariupol and volnovakha were halted amid russian shelling.
azov surrender
Barsik: 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


"'surrender is not an option': azov battalion commander in plea for help to escape mariupol".the guardian.
